In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import plotly.offline as pyo
from dateutil import parser
from Strategies import AmipyNF
from kiteconnect import KiteConnect
from Brokers import apikey

import pandas as pd
from kiteconnect import KiteConnect
import undetected_chromedriver as uc
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from datetime import datetime
import time, pyotp

import logging
from kiteconnect import KiteTicker
from kiteconnect import KiteConnect
from Brokers import apikey
from datetime import datetime
import time

In [2]:
acctkn_file = r'Brokers\acc_token.txt'
reqtkn_file = r'Brokers\req_token.txt'
kite_access_token = open(acctkn_file,'r').read()
kite_request_token = open(reqtkn_file,'r').read()
print(kite_access_token)

OgsugX2hN7497sGgE1Kf26RqAQojS8JU


In [3]:
expiry_date = '2023-05-25'
tokens = [256265]
strike_prc = 18200

from_date = '2023-05-17'
to_date = '2023-05-19'
interval = 'minute'

In [4]:
# for given strike price and option type, get instrument token from instruments.csv

instruments_df = pd.read_csv(r'Brokers\instruments.csv')

instruments_df = instruments_df[
    ["instrument_token", "tradingsymbol", "name", "exchange", "lot_size","instrument_type","expiry","strike"]
]

# from instruments dataframe, get all instruments for 'NFO' exchange, name =='NIFTY',expiry date is 4thMay2023, strike value between 17000 to 19000
nfo_ins_df = instruments_df[
    (instruments_df["exchange"] == "NFO")
    & (instruments_df["name"] == "NIFTY")
    & (instruments_df["expiry"] == expiry_date)
    & (instruments_df["strike"] == strike_prc)
]    
tokens.append(nfo_ins_df['instrument_token'].values[0])
tokens.append(nfo_ins_df['instrument_token'].values[1])
tokens

[256265, 18845442, 18845698]

In [5]:
kite = KiteConnect(apikey.kite_api_key)
kite.set_access_token(kite_access_token)

In [6]:
holidays = ['2023-05-01', '2023-06-16']  # Add all trading holidays here

# Remove the holidays from the resampled dataframe
def remove_holiday_data(df, holidays):
    
    print("Before",len(df))
    
    # Convert the holidays list into a set for faster lookup
    holidays_set = set(pd.to_datetime(holiday).date() for holiday in holidays)

    # Create a boolean mask to identify rows with dates not in the holidays set
    mask = [date not in holidays_set for date in df.index.date]

    # Apply the mask to the DataFrame to remove rows with dates in the holidays set
    df = df[mask]
    
    print("After",len(df))
    
    return df

In [7]:
# Supertrend params
Heikin_Ashi_MA_period = 13
Supertrend_period = 7
Supertrend_multiplier = 18
EMA_period = 324

In [8]:
def moving_average(df, HA_MA_period):
    MA_open = pd.Series(df['open'].rolling(HA_MA_period, min_periods=HA_MA_period).mean(), name='MA_open')
    MA_high = pd.Series(df['high'].rolling(HA_MA_period, min_periods=HA_MA_period).mean(), name='MA_high')
    MA_low = pd.Series(df['low'].rolling(HA_MA_period, min_periods=HA_MA_period).mean(), name='MA_low')
    MA_close = pd.Series(df['close'].rolling(HA_MA_period, min_periods=HA_MA_period).mean(), name='MA_close')
    
    df = df.join(MA_open).join(MA_close).join(MA_high).join(MA_low)
    return df

In [9]:
def HeikinAshi(ma_df):
    ha_df = pd.DataFrame(index=ma_df.index)

    ha_df['ha_close'] = (ma_df['MA_open'] + ma_df['MA_high'] + ma_df['MA_high'] + ma_df['MA_close']) / 4

    ha_df['ha_open'] = ha_df['ha_close'].shift(1).ewm(alpha=0.5, adjust=False).mean()

    ha_df['ha_high'] = ma_df['MA_high'].combine(ha_df['ha_close'], max).combine(ha_df['ha_open'], max)

    ha_df['ha_low'] = ma_df['MA_close'].combine(ha_df['ha_close'], min).combine(ha_df['ha_open'], min)

    ma_df = pd.concat([ma_df, ha_df], axis=1)
    

In [10]:
def atr(df, period):
    true_range = pd.Series(index=df.index)
    for i in range(1, len(df)):
        high_low = df['high'][i] - df['low'][i]
        high_prev_close = abs(df['high'][i] - df['close'][i - 1])
        low_prev_close = abs(df['low'][i] - df['close'][i - 1])
        true_range[i] = max(high_low, high_prev_close, low_prev_close)
    return true_range.ewm(span=period, adjust=False).mean()

In [20]:
def supertrend(ma_df):
    # Supertrend params
    global Heikin_Ashi_MA_period
    global Supertrend_period
    global Supertrend_multiplier
    global EMA_period
    # Calculate Average True Range (ATR)
    average_true_range = atr(ma_df, Supertrend_period)
    ma_df['ATR'] = average_true_range

    # Calculate Heikin-Ashi values
    ha_close = (ma_df['open'] + ma_df['high'] + ma_df['low'] + ma_df['close']) / 4
    ha_open = (ma_df['open'].shift(1) + ma_df['close'].shift(1)) / 2
    ha_high = ma_df[['high', 'open', 'close']].max(axis=1)
    ha_low = ma_df[['low', 'open', 'close']].min(axis=1)

    # Calculate Up and Dn lines
    up = (ha_high + ha_low) / 2 + Supertrend_multiplier * average_true_range
    dn = (ha_high + ha_low) / 2 - Supertrend_multiplier * average_true_range

    # Initialize variables
    trend = np.zeros(len(ma_df))
    trend[0] = 1
    trend_up = pd.Series(index=ma_df.index)
    trend_down = pd.Series(index=ma_df.index)

    change_of_trend = 0
    flag = flagh = 0

    # calulating change of trend
    for i in range(1, len(ma_df)):
        change_of_trend = 0
        
        if ha_close[i] > up[i - 1]:
            trend[i] = 1
            if trend[i-1] == -1:
                change_of_trend = 1
        elif ha_close[i] < dn[i - 1]:
            trend[i] = -1
            if trend[i-1] == 1:
                change_of_trend = 1
        else:
            trend[i] = trend[i - 1]

        if trend[i] < 0 and trend[i-1] > 0:
            flag = 1
        else:
            flag = 0

        if trend[i] > 0 and trend[i-1] < 0:
            flagh = 1
        else:
            flagh = 0

        if trend[i] > 0 and dn[i] < dn[i - 1]:
            dn[i] = dn[i - 1]

        if trend[i] < 0 and up[i] > up[i - 1]:
            up[i] = up[i - 1]

        if flag == 1:
            up[i] = (ha_high[i] + ha_low[i]) / 2 + Supertrend_multiplier * average_true_range[i]

        if flagh == 1:
            dn[i] = (ha_high[i] + ha_low[i]) / 2 - Supertrend_multiplier * average_true_range[i]

        trend_up[i] = dn[i] if trend[i] > 0 else np.nan
        trend_down[i] = up[i] if trend[i] < 0 else np.nan

        if change_of_trend == 1:
            if trend[i] == 1:
                trend_up[i - 1] = trend_down[i - 1]
            elif trend[i] == -1:
                trend_down[i - 1] = trend_up[i - 1]

    # Create a new DataFrame with calculated values
    result = ma_df.copy()

    # result['strike_prc'] = ma_df[]    
    result['ATR'] = average_true_range
    result["EMA"] = result["close"].ewm(span=EMA_period, adjust=False).mean()
    result['Up'] = up
    result['Dn'] = dn
    result['Trend'] = trend
    result['TrendUp'] = trend_up
    result['TrendDown'] = trend_down
    result['TrendSL'] = np.where(result['Trend'] == 1, result['TrendUp'], result['TrendDown'])
    #result.set_index('date', inplace=True)
    columns_order = ['close', 'MA_open', 'MA_close', 'MA_high', 'MA_low','ATR','Up', 'Dn', 'Trend', 'TrendUp',
                    'TrendDown', 'TrendSL','EMA','open', 'high', 'low']

    result = result[columns_order]
    result.to_csv(f'amipy_result_dev.csv', index=True)
    return result

In [12]:
def genSignals(resultdf):
    # Time params
    entry_time = pd.Timestamp("09:21:00").time()
    last_buy_time = pd.Timestamp("14:50:00").time()
    sqroff_time = pd.Timestamp("14:58:00").time()     
    counter = 0
    signals = []
    trade_no = 1
    current_position = None
    global last_signal_minute, tokens,trade_state_df
    trade_state = {'in_trade': False, 'trade_points': 0, 'TrendSL': 0, 'close': 0, 'TradeEntryPrice': 0, 'SL_points': 0}

    long_indices = []
    longcover_indices = []
    short_indices = []
    shortcover_indices = []
    

    def long(df, i):
        return (df.loc[df.index[i], "Trend"] == 1) & (df.loc[df.index[i], "close"] > df.loc[df.index[i], "EMA"]) & (df.index[i].time() > entry_time) & (df.index[i].time() < sqroff_time) & (df.index[i].time() < last_buy_time)

    def longcover(df, i):
        return (df.loc[df.index[i], "Trend"] == -1) | (df.index[i].time() > sqroff_time)

    def short(df, i):
        return (df.loc[df.index[i], "Trend"] == -1) & (df.loc[df.index[i], "close"] < df.loc[df.index[i], "EMA"]) & (df.index[i].time() > entry_time) & (df.index[i].time() < sqroff_time) & (df.index[i].time() < last_buy_time)

    def shortcover(df, i):
        return (df.loc[df.index[i], "Trend"] == 1) | (df.index[i].time() > sqroff_time)

    
    for i in range(1, len(resultdf)):
        current_time = resultdf.index[i]
        current_minute = current_time.floor('T')
        current_close = resultdf.loc[current_time, 'close']
        trend_sl = resultdf.loc[current_time, 'TrendSL']

        if current_position is None:
            if long(resultdf, i):
                counter += 1
                current_position = 'Long'
                long_indices.append(resultdf.index[i])
                
                signal = {
                    # 'Strike_Price': resultdf.index[i]['strike_prce_ce'],
                    'Trade_No': trade_no,
                    'Trade_Type': current_position,
                    'Date': current_time.date(),
                    'TradeEntryTime': current_time.time(),
                    'TradeEntryPrice': current_close,
                }
                signals.append(signal)
                trade_state['in_trade'] = True
                trade_state['TradeEntryPrice'] = current_close
                trade_state['TrendSL'] = trend_sl
                trade_state['SL_points'] = current_close - trend_sl
                signals_df = pd.DataFrame(signals)
                signals_df.to_csv('amiNF_trd_sig_liv.csv', index=True)
                        

            elif short(resultdf, i):
                current_position = 'Short'
                short_indices.append(resultdf.index[i])

                signal = {
                    # 'Strike_Price': resultdf.index[i]['strike_prce_ce'],
                    'Trade_No': trade_no,
                    'Trade_Type': current_position,
                    'Date': current_time.date(),
                    'TradeEntryTime': current_time.time(),
                    'TradeEntryPrice': current_close,
                }
                signals.append(signal)
                trade_state['in_trade'] = True
                trade_state['TradeEntryPrice'] = current_close
                trade_state['TrendSL'] = trend_sl
                trade_state['SL_points'] = trend_sl - current_close
                signals_df = pd.DataFrame(signals)
                signals_df.to_csv('amiNF_trd_sig_liv.csv', index=True)
        else:
            if current_position == 'Long' and longcover(resultdf, i):
                longcover_indices.append(resultdf.index[i])

                last_signal = signals[-1]
                last_signal.update({
                    'TradeExitTime': current_time.time(),
                    'TradeExitPrice': current_close,
                })
                last_signal['NetTradePoints'] = last_signal['TradeExitPrice'] - last_signal['TradeEntryPrice']
                trade_no += 1
                trade_state['in_trade'] = False
                trade_state['trade_points'] = current_close - trade_state['TradeEntryPrice']
                trade_state['TrendSL'] = 0
                trade_state['SL_points'] = 0
                signals_df = pd.DataFrame(signals)
                signals_df.to_csv('amiNF_trd_sig_liv.csv', index=True)
                current_position = None
                

            elif current_position == 'Short' and shortcover(resultdf, i):
                shortcover_indices.append(resultdf.index[i])
                
                last_signal = signals[-1]
                last_signal.update({
                    'TradeExitTime': current_time.time(),
                    'TradeExitPrice': current_close,
                })
                last_signal['NetTradePoints'] = last_signal['TradeEntryPrice'] - last_signal['TradeExitPrice']
                trade_no += 1
                trade_state['in_trade'] = False
                trade_state['trade_points'] = trade_state['TradeEntryPrice'] - current_close
                trade_state['TrendSL'] = 0
                trade_state['SL_points'] = 0
                signals_df = pd.DataFrame(signals)
                signals_df.to_csv('amiNF_trd_sig_liv.csv', index=True)
                current_position = None                   
        
        trade_state_df = pd.DataFrame([trade_state])  # convert dictionary to DataFrame
        trade_state_df.to_csv('trade_state.csv', index=False)  # write DataFrame to CSV
        
        resultdf['LongSignal'] = 0
        resultdf['ShortSignal'] = 0
        resultdf['LongCoverSignal'] = 0
        resultdf['ShortCoverSignal'] = 0

        resultdf.loc[long_indices, 'LongSignal'] = 1
        resultdf.loc[short_indices, 'ShortSignal'] = 1
        resultdf.loc[longcover_indices, 'LongCoverSignal'] = 1
        resultdf.loc[shortcover_indices, 'ShortCoverSignal'] = 1
        resultdf.to_csv(f'amipy_genSignals.csv', index=True)

    return signals_df,resultdf

In [13]:
nf_hist_data = kite.historical_data(tokens[0], from_date, to_date, interval)
call_hist_data = kite.historical_data(tokens[1], from_date, to_date, interval)
put_hist_data = kite.historical_data(tokens[2], from_date, to_date, interval)

# Convert historical_data to pandas dataframe
nf_df = pd.DataFrame(nf_hist_data)
nf_df = nf_df.set_index('date')

# add instrument token variable as column to nf_df
nf_df['instrument_token'] = tokens[0]

nf_call_df = pd.DataFrame(call_hist_data)
nf_call_df = nf_call_df.set_index('date')
nf_call_df['instrument_token'] = tokens[1]

nf_put_df = pd.DataFrame(put_hist_data)
nf_put_df = nf_put_df.set_index('date')
nf_put_df['instrument_token'] = tokens[2]

trans_df = nf_call_df.add(nf_put_df, fill_value=0)

# combine combined_df and nf_df renaming columns of nf_df to 'nf_' + column name
trans_df = pd.concat([trans_df, nf_df.add_prefix('nf_')], axis=1)

# Resample the DataFrame at 1-minute intervals
resampled_df = trans_df.resample("1T").asfreq()
resampled_df = resampled_df.interpolate(method="time")

# Drop all rows from resampled_df when date is saturday or sunday
resampled_df = resampled_df[resampled_df.index.dayofweek < 5]

# Drop all rows from resampled_df when time is before 9:15 or after 15:30
resampled_df = resampled_df.between_time("09:15", "15:30")

resampled_df = remove_holiday_data(resampled_df,holidays)

resampled_df = resampled_df.drop(['volume','nf_volume','nf_instrument_token','nf_high','nf_low','nf_open'], axis=1)


resampled_df.to_csv(f'ce_{tokens[1]}_df.csv', index=True)


Before 1127
After 1127


In [17]:
ma_df = moving_average(resampled_df, Heikin_Ashi_MA_period)
# print(ma_df)
merged_df2 = supertrend(ma_df)
results = genSignals(merged_df2)
signalsdf = results[0]
merged_df2 = results[1]
merged_df2.to_csv(f'amipy_merged_df2.csv', index=True)


In [18]:
def plotly_plot(merged_df2):
    #global trade_state_df
    # Define the custom shapes (background color) for each day
    shapes = []
    colors = ["white", "#C23B22", "#5F9EA0", "#F5DEB3", "black"]
    holidayz = [datetime.strptime(date_str, '%Y-%m-%d').date() for date_str in ['2023-05-01', '2023-06-16']]

    # print datatypes of resultdf columns including index
    for i, date in enumerate(merged_df2.index):
        if date.hour == 9 and date.minute == 15:
            weekday = date.weekday()
            if weekday < 5:  # Only for Monday to Friday
                if date.date() not in holidayz:  # Check if the date is not a holiday
                    shape = {
                        "type": "rect",
                        "xref": "x",
                        "yref": "paper",
                        "x0": date,
                        "x1": date + pd.Timedelta(hours=6, minutes=15),
                        "y0": 0,
                        "y1": 1,
                        "fillcolor": colors[weekday],
                        "opacity": 0.3,
                        "line": {"width": 0},
                        "layer": "below",
                    }
                    shapes.append(shape)
                    
    holiday_rangebreaks = [{"bounds": [holiday.strftime('%Y-%m-%d'), (holiday + pd.Timedelta(days=1)).strftime('%Y-%m-%d')], "pattern": ''} for holiday in holidayz]

    # Create a layout with custom styling, including background colors and shapes
    layout = go.Layout(
        title="AmiPy_NF",
        plot_bgcolor="black",
        height=800,
        xaxis=dict(
            gridcolor="rgba(128, 128, 128, 0.1)",
            showgrid=True,
            tickformat="%Y-%m-%d %H:%M",
            dtick="60 * 60 * 1000",  # 1 hour in milliseconds
            rangebreaks=[
                dict(bounds=["sat", "mon"]),
                dict(bounds=[0, 9.25], pattern="hour"),
                dict(bounds=[15.5, 24], pattern="hour"),
            ],
        ),
        yaxis=dict(
            gridcolor="rgba(128, 128, 128, 0.1)",
            showgrid=True,
            automargin=True,
            autorange=True,
        ),
        # Add the custom shapes to the layout
        shapes=shapes,
    )

    fig = go.Figure()

    fig.add_trace(go.Scatter(x=merged_df2.index, y=merged_df2['close'],
                            mode='lines', name='Closing Price', line=dict(color='skyblue', dash='dot')))
    
        # Filter the data for uptrends and downtrends
    uptrend_df = merged_df2[merged_df2['Trend'] == 1]
    downtrend_df = merged_df2[merged_df2['Trend'] == -1]

    # Plot the uptrend data with a green line
    for i in range(len(uptrend_df) - 1):
        if uptrend_df.index[i+1] - uptrend_df.index[i] == pd.Timedelta(minutes=1):
            fig.add_trace(go.Scatter(x=[uptrend_df.index[i], uptrend_df.index[i+1]], y=[uptrend_df.iloc[i]['TrendSL'], uptrend_df.iloc[i+1]['TrendSL']],
                                    mode='lines', name='SuperTrend Uptrend', line=dict(color='green'), showlegend=False if i != 0 else True))

    # Plot the downtrend data with a red line
    for i in range(len(downtrend_df) - 1):
        if downtrend_df.index[i+1] - downtrend_df.index[i] == pd.Timedelta(minutes=1):
            fig.add_trace(go.Scatter(x=[downtrend_df.index[i], downtrend_df.index[i+1]], y=[downtrend_df.iloc[i]['TrendSL'], downtrend_df.iloc[i+1]['TrendSL']],
                                    mode='lines', name='SuperTrend Downtrend', line=dict(color='red'), showlegend=False if i != 0 else True))

    # Add the custom shapes to the layout
    layout.shapes = shapes

    # Update x-axis tick format
    layout.xaxis.tickformat = "%Y-%m-%d %H:%M"
    layout.xaxis.hoverformat = "%Y-%m-%d %H:%M"

    fig.add_trace(go.Scatter(x=merged_df2[merged_df2['LongSignal'] == 1].index, y=merged_df2[merged_df2['LongSignal'] == 1]['close'], mode='markers', name='Long', marker=dict(color='green', symbol='triangle-up', size=15)))
    fig.add_trace(go.Scatter(x=merged_df2[merged_df2['LongCoverSignal'] == 1].index, y=merged_df2[merged_df2['LongCoverSignal'] == 1]['close'], mode='markers', name='Long Cover', marker=dict(color='green', symbol='star', size=15)))
    fig.add_trace(go.Scatter(x=merged_df2[merged_df2['ShortSignal'] == 1].index, y=merged_df2[merged_df2['ShortSignal'] == 1]['close'], mode='markers', name='Short Signal', marker=dict(color='red', symbol='triangle-down', size=15)))
    fig.add_trace(go.Scatter(x=merged_df2[merged_df2['ShortCoverSignal'] == 1].index, y=merged_df2[merged_df2['ShortCoverSignal'] == 1]['close'], mode='markers', name='Short Cover Signal', marker=dict(color='red', symbol='star', size=15)))

    fig.update_layout(layout)
    
    pyo.plot(fig)   
  
    return fig

In [19]:
plotly_plot(merged_df2)